(all code in this example are from the notebook : https://www.kaggle.com/abhinand05/bert-for-humans-tutorial-baseline/notebook by [abhinand05](https://www.kaggle.com/abhinand05))

# Problematic

The code below present a machine learning model for text understanding. We use a fine-tuning mecanism of BERT model to get a response to a generic question about an article.
It's an extractive approach that extract the response from the article using a start and end position. two responses are possible : a large and small one.

# Code Implementation on TSF 2.0

Qute from the original notebook : 

> In this notebook, we'll be using the Bert baseline for Tensorflow to create predictions for the Natural Questions test set. Note that this uses a model that has already been pre-trained - we're only doing inference here. A GPU is required, and this should take between 1-2 hours to run.
> 
> The original script can be found [here](https://github.com/google-research/language/blob/master/language/question_answering/bert_joint/run_nq.py).
> The supporting modules were drawn from the [official Tensorflow model repository](https://github.com/tensorflow/models/tree/master/official). The bert-joint-baseline data is described [here](https://github.com/google-research/language/tree/master/language/question_answering/bert_joint).


The most code below is from the original notefook (link on thte top), However I a review it, add comment, fixe issues and make it more readebale.

In [ ]:
"""
Import all needed Python Package
"""
import numpy as np
import pandas as pd
import tensorflow as tf


# this is translation of BERT in TensorFlow 2.0, made by dimitre oliveira (https://www.kaggle.com/dimitreoliveira)
import tf2_0_baseline_w_bert_translated_to_tf2_0 as tf2baseline # Oliviera's script

import bert_modeling as modeling
import bert_optimization as optimization
import bert_tokenization as tokenization

import json
import absl
import sys

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)

"""
Delete All TF Flags that are already defined
"""        
del_all_flags(absl.flags.FLAGS)


"""
Define TF flags, that represent variables used by our system and model

Note :  All thoses flags can be changed as desired during the code
"""

flags = absl.flags

flags.DEFINE_string(
    "bert_config_file", "/kaggle/input/bertjointbaseline/bert_config.json",
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

flags.DEFINE_string("vocab_file", "/kaggle/input/bertjointbaseline/vocab-nq.txt",
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string(
    "output_dir", "outdir",
    "The output directory where the model checkpoints will be written.")

flags.DEFINE_string("train_precomputed_file", None,
                    "Precomputed tf records for training.")

flags.DEFINE_integer("train_num_precomputed", None,
                     "Number of precomputed tf records for training.")

flags.DEFINE_string(
    "output_prediction_file", "predictions.json",
    "Where to print predictions in NQ prediction format, to be passed to"
    "natural_questions.nq_eval.")

flags.DEFINE_string(
    "init_checkpoint", "/kaggle/input/bertjointbaseline/bert_joint.ckpt",
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_integer(
    "max_seq_length", 384,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_integer(
    "doc_stride", 128,
    "When splitting up a long document into chunks, how much stride to "
    "take between chunks.")

flags.DEFINE_integer(
    "max_query_length", 64,
    "The maximum number of tokens for the question. Questions longer than "
    "this will be truncated to this length.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")

flags.DEFINE_bool("do_predict", True, "Whether to run eval on the dev set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("predict_batch_size", 8,
                     "Total batch size for predictions.")

flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

flags.DEFINE_integer("save_checkpoints_steps", 1000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_integer(
    "n_best_size", 20,
    "The total number of n-best predictions to generate in the "
    "nbest_predictions.json output file.")

flags.DEFINE_integer(
    "verbosity", 1, "How verbose our error messages should be")

flags.DEFINE_integer(
    "max_answer_length", 30,
    "The maximum length of an answer that can be generated. This is needed "
    "because the start and end predictions are not conditioned on one another.")

flags.DEFINE_float(
    "include_unknowns", -1.0,
    "If positive, probability of including answers of type `UNKNOWN`.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")
flags.DEFINE_bool("use_one_hot_embeddings", False, "Whether to use use_one_hot_embeddings")

absl.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

flags.DEFINE_bool(
    "verbose_logging", False,
    "If true, all of the warnings related to data processing will be printed. "
    "A number of warnings are expected for a normal NQ evaluation.")

flags.DEFINE_boolean(
    "skip_nested_contexts", True,
    "Completely ignore context that are not top level nodes in the page.")

flags.DEFINE_integer("task_id", 0,
                     "Train and dev shard to read from and write to.")

flags.DEFINE_integer("max_contexts", 48,
                     "Maximum number of contexts to output for an example.")

flags.DEFINE_integer(
    "max_position", 50,
    "Maximum context position for which to generate special tokens.")


## Special flags - do not change

flags.DEFINE_string(
    "predict_file", "/kaggle/input/tensorflow2-question-answering/simplified-nq-test.jsonl",
    "NQ json for predictions. E.g., dev-v1.1.jsonl.gz or test-v1.1.jsonl.gz")
flags.DEFINE_boolean("logtostderr", True, "Logs to stderr")
flags.DEFINE_boolean("undefok", True, "it's okay to be undefined")
flags.DEFINE_string('f', '', 'kernel')
flags.DEFINE_string('HistoryManager.hist_file', '', 'kernel')

FLAGS = flags.FLAGS
FLAGS(sys.argv) # Parse the flags

**Next we will implement our fine-tuning of BERT to resolve our initial issue**

In [ ]:
"""
1. Set up BERT Model
"""

# get configuration of BERT from the file "bert_config.json"
bert_config = modeling.BertConfig.from_json_file(FLAGS.bert_config_file)

tf2baseline.validate_flags_or_throw(bert_config)

tf.io.gfile.makedirs(FLAGS.output_dir)

run_config = tf.estimator.RunConfig(
    model_dir=FLAGS.output_dir,
    save_checkpoints_steps=FLAGS.save_checkpoints_steps)

# get a tokenizer, based on BERT Vocabs file
tokenizer = tokenization.FullTokenizer(
    vocab_file=FLAGS.vocab_file, do_lower_case=FLAGS.do_lower_case)

num_train_steps = None
num_warmup_steps = None

# initialise BERT MODEL (Question : How the model is uploaded to model_fn)
model_fn = tf2baseline.model_fn_builder(
    bert_config=bert_config,
    init_checkpoint=FLAGS.init_checkpoint,
    learning_rate=FLAGS.learning_rate,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=FLAGS.use_tpu,
    use_one_hot_embeddings=FLAGS.use_one_hot_embeddings)

# initialise estimator
estimator = tf.estimator.Estimator(
    model_fn=model_fn,
    config=run_config,
    params={'batch_size':FLAGS.train_batch_size})


In [ ]:
"""
2. Read in the test set
"""

print("FLAGS.predict_file : ", FLAGS.predict_file)

eval_examples = tf2baseline.read_nq_examples(
      input_file=FLAGS.predict_file, is_training=False) 


random_examples = eval_examples[np.random.randint(len(eval_examples) -1)]

print('Random Example / id', random_examples.example_id)
print('Random Example / qas_id', random_examples.qas_id)
print('Random Example / questions', random_examples.questions)
print('Random Example / doc_tokens', random_examples.doc_tokens[:20])
print('Random Example / doc_tokens_map', random_examples.doc_tokens_map[:20])
print('Random Example / answer', random_examples.answer)
print('Random Example / start_position', random_examples.start_position)
print('Random Example / end_position', random_examples.end_position)


In [ ]:
"""
3. Run it past the pre-built Bert model to create embeddings
"""

def append_feature(feature):
    eval_features.append(feature)
    eval_writer.process_feature(feature)
    
eval_writer = tf2baseline.FeatureWriter(
      filename=os.path.join(FLAGS.output_dir, "eval.tf_record"),
      is_training=False)

eval_features = []

num_spans_to_ids = tf2baseline.convert_examples_to_features(
      examples=eval_examples,
      tokenizer=tokenizer,
      is_training=False,
      output_fn=append_feature)

eval_writer.close()

random_features = eval_features[np.random.randint(len(eval_features) -1)]

print('Random Eval Feature / unique_id', random_features.unique_id)
print('Random Eval Feature / example_index', random_features.example_index)
print('Random Eval Feature / doc_span_index', random_features.doc_span_index)
print('Random Eval Feature / tokens', random_features.tokens[:10])
print('Random Eval Feature / token_to_orig_map', list(random_features.token_to_orig_map)[:10] )
print('Random Eval Feature / token_is_max_context', list(random_features.token_is_max_context)[:10])
print('Random Eval Feature / input_ids', random_features.input_ids[:10])
print('Random Eval Feature / input_mask', random_features.input_mask[:10])
print('Random Eval Feature / segment_ids', random_features.segment_ids[:10])
print('Random Eval Feature / start_position', random_features.start_position)
print('Random Eval Feature / end_position', random_features.end_position)
print('Random Eval Feature / answer_text', random_features.answer_text)
print('Random Eval Feature / answer_type', random_features.answer_type)


In [ ]:
"""
4. Use embeddings to fine-tuning BERT Model
"""

eval_filename = eval_writer.filename

print("***** Running predictions *****")
print(f"  Num orig examples = %d" % len(eval_examples))
print(f"  Num split examples = %d" % len(eval_features))
print(f"  Batch size = %d" % FLAGS.predict_batch_size)


#for spans, ids in num_spans_to_ids.items():
#    print(f"  Num split into %d = %d" % (spans, len(ids)))

predict_input_fn = tf2baseline.input_fn_builder(
      input_file=eval_filename,
      seq_length=FLAGS.max_seq_length,
      is_training=False,
      drop_remainder=False)

all_results = []

for result in estimator.predict( predict_input_fn, yield_single_examples=True):
    if len(all_results) % 1000 == 0:
      print("Processing example: %d" % (len(all_results)))

    unique_id = int(result["unique_ids"])
    start_logits = [float(x) for x in result["start_logits"].flat]
    end_logits = [float(x) for x in result["end_logits"].flat]
    answer_type_logits = [float(x) for x in result["answer_type_logits"].flat]

    all_results.append(
        tf2baseline.RawResult(
            unique_id=unique_id,
            start_logits=start_logits,
            end_logits=end_logits,
            answer_type_logits=answer_type_logits))


In [ ]:
'''
5. Make predictions
'''

candidates_dict = tf2baseline.read_candidates(FLAGS.predict_file)

raw_dataset = tf.data.TFRecordDataset(eval_filename)

eval_features = []
for raw_record in raw_dataset:
    eval_features.append(tf.train.Example.FromString(raw_record.numpy()))
    

nq_pred_dict = tf2baseline.compute_pred_dict(candidates_dict, eval_features,
                                   [r._asdict() for r in all_results])


In [ ]:
'''
5. Display random predictions
'''

def create_short_answer(entry):
    # if entry["short_answers_score"] < 1.5:
    #     return ""
    
    answer = []    
    for short_answer in entry["short_answers"]:
        if short_answer["start_token"] > -1:
            answer.append(str(short_answer["start_token"]) + ":" + str(short_answer["end_token"]))
    if entry["yes_no_answer"] != "NONE":
        answer.append(entry["yes_no_answer"])
    return " ".join(answer)

def create_long_answer(entry):
   # if entry["long_answer_score"] < 1.5:
   # return ""

    answer = []
    if entry["long_answer"]["start_token"] > -1:
        answer.append(str(entry["long_answer"]["start_token"]) + ":" + str(entry["long_answer"]["end_token"]))
    return " ".join(answer)

predict_values = list(nq_pred_dict.values())
random_index = np.random.randint(len(predict_values))

start_short_answer, end_short_answer = map(int, create_short_answer(predict_values[random_index]).split(':'))
start_long_answer, end_long_answer = map(int, create_long_answer(predict_values[random_index]).split(':'))

prediction = predict_values[random_index]

for ex in eval_examples:
    if( ex.example_id == int(prediction['example_id']) ) :
        print('Question : ', ex.questions)
        print( '*' * 50)
        print('Short answer : ', ' '.join(ex.doc_tokens[start_short_answer:end_short_answer]))
        print('Long answer : ', ' '.join(ex.doc_tokens[start_long_answer:end_long_answer]))
        print( '*' * 50)
        print('Document : ', ' '.join(ex.doc_tokens))
        break


<font size=4 color='#57467B'>If you find this kernel useful ! Please Give it an UPVOTE  :) </font>